In [ ]:
os.mkdir('/content/folds')
for i in range(0,10):
  os.mkdir('/content/folds/fold'+str(i))
  os.mkdir('/content/folds/fold'+str(i)+'/train')
  os.mkdir('/content/folds/fold'+str(i)+'/test')
  for j in os.listdir('/content/datasets/CK+48'):
    os.mkdir('/content/folds/fold'+str(i)+'/train/'+j)
    os.mkdir('/content/folds/fold'+str(i)+'/test/'+j)

In [ ]:
for i in range(0,10):
  fold_path = '/content/folds/fold' + str(i)
  test_fold = '/content/folds/fold' + str(i) + '/test'
  train_fold = '/content/folds/fold' + str(i) + '/train'
  for j in os.listdir(train_fold):
    emotion_train_fold = train_fold + '/' + j
    emotion_test_fold = test_fold + '/' + j
    emotion_source = '/content/datasets/CK+48/' + j
    length = len(os.listdir(emotion_source))
    initial_size = int(i*length/10)
    final_size = int((i+1)*length/10)
    files = []
    for k in os.listdir(emotion_source):
      path = emotion_source + '/' + k 
      files.append(k)
    testing_set = files[initial_size:final_size]
    training_set = []
    for n in files:
      if n not in testing_set:
        training_set.append(n)
    for filename in training_set:
      src = emotion_source + '/' + filename
      des = emotion_train_fold + '/' + filename
      copyfile(src,des)
    for filename in testing_set:
      src = emotion_source + '/' + filename
      des = emotion_test_fold + '/' + filename
      copyfile(src,des)

In [ ]:
sum_acc=0
for i in range(0,10):
  fold_path = '/content/folds/fold' + str(i)
  TRAINING_DIR = train_fold = '/content/folds/fold' + str(i) + '/train'
  train_datagen = ImageDataGenerator(rescale=1./255,
      horizontal_flip=True,
      rotation_range=2,                          
      preprocessing_function=preprocessing
      )
  train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=50,
                                                    class_mode='categorical',
                                                    target_size=(100,100),
                                                    shuffle=True,
                                                    color_mode='grayscale')

  VALIDATION_DIR = '/content/folds/fold' + str(i) + '/test'
  validation_datagen = ImageDataGenerator(
    rescale=1./255,
      horizontal_flip=True,
      rotation_range=2,
      preprocessing_function=preprocessing
      )
  validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=50,
                                                              class_mode='categorical',
                                                              target_size=(100,100),
                                                              shuffle=True,
                                                    color_mode='grayscale')
  validation_generator.shuffle = False
  validation_generator.index_array = None
  model = CNN_model_with_1024_neurons(7)
  model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])
  history = model.fit(train_generator,epochs=10,batch_size=50,shuffle=True)
  test_loss,test_acc=model.evaluate(validation_generator)
  sum_acc=sum_acc+test_acc
avg_acc=sum_acc/10
print(avg_acc)